In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier

# Configuración de rutas de datos
ruta_base = 'data/data_format1/data_format1/'
ruta_train = ruta_base + 'train_format1.csv'
ruta_test = ruta_base + 'test_format1.csv'
ruta_info_usuarios = ruta_base + 'user_info_format1.csv'
ruta_log_usuarios = ruta_base + 'user_log_format1.csv'

# Cargar datos
datos_train = pd.read_csv(ruta_train)
datos_test = pd.read_csv(ruta_test)
info_usuarios = pd.read_csv(ruta_info_usuarios)
log_usuarios = pd.read_csv(ruta_log_usuarios)

# Fusionar y marcar datos de entrenamiento y prueba
datos_train['tipo'] = 'train'
datos_test['tipo'] = 'test'
conjunto_datos = pd.concat([datos_train, datos_test.drop('prob', axis=1)], ignore_index=True)

# Optimizar tipos de datos para reducir uso de memoria
log_usuarios = log_usuarios.astype({
    'user_id': 'int32',
    'item_id': 'int32',
    'cat_id': 'int16',
    'seller_id': 'int16'
})
log_usuarios.rename(columns={'seller_id': 'merchant_id'}, inplace=True)
log_usuarios['brand_id'] = log_usuarios['brand_id'].fillna(0).astype('int16')
log_usuarios['action_type'] = log_usuarios['action_type'].astype('int8')

# Transformar fechas en 'time_stamp' a días transcurridos desde la fecha mínima
fecha_base = pd.to_datetime(log_usuarios['time_stamp'], format='%m%d').min()
log_usuarios['time_stamp'] = (pd.to_datetime(log_usuarios['time_stamp'], format='%m%d') - fecha_base).dt.days
log_usuarios['time_stamp'] = log_usuarios['time_stamp'].astype('int16')

# Imputar valores nulos y ajustar tipos en info_usuarios
info_usuarios['age_range'].fillna(0, inplace=True)
info_usuarios['gender'].fillna(2, inplace=True)
info_usuarios = info_usuarios.astype({'age_range': 'int8', 'gender': 'int8'})

# Codificación de variable categórica 'age_range'
encoder = OneHotEncoder(sparse_output=False)
edad_codificada = encoder.fit_transform(info_usuarios[['age_range']])

# Crear un DataFrame de las columnas codificadas con nombres personalizados para evitar problemas de nombres
nombres_columnas = [f"age_{int(cat)}" for cat in encoder.categories_[0]]
edad_codificada_df = pd.DataFrame(edad_codificada, columns=nombres_columnas)

# Alinear el índice de 'edad_codificada_df' con 'info_usuarios' antes de concatenar
edad_codificada_df.index = info_usuarios.index

# Concatenar las columnas codificadas al DataFrame de info_usuarios
info_usuarios = pd.concat([info_usuarios, edad_codificada_df], axis=1)

C:\Users\daher\AppData\Local\Temp\ipykernel_12824\3578034257.py:43: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  info_usuarios['age_range'].fillna(0, inplace=True)
C:\Users\daher\AppData\Local\Temp\ipykernel_12824\3578034257.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

In [2]:
# Agrupar por usuario y calcular características de interacción
user_agg = log_usuarios.groupby('user_id').agg(
    items_distintos=('item_id', 'nunique'),
    categorias_distintas=('cat_id', 'nunique'),
    merchants_distintos=('merchant_id', 'nunique'),
    marcas_distintas=('brand_id', 'nunique'),
    dias_distintos=('time_stamp', 'nunique'),
    dia_max=('time_stamp', 'max'),
    dia_min=('time_stamp', 'min'),
    clicks=('action_type', lambda x: (x == 0).sum()),
    carritos=('action_type', lambda x: (x == 1).sum()),
    compras=('action_type', lambda x: (x == 2).sum()),
    favoritos=('action_type', lambda x: (x == 3).sum())
).reset_index()

In [6]:
# Crear ratios de interacción y otras métricas
user_agg['rango_dias'] = user_agg['dia_max'] - user_agg['dia_min']
user_agg['ratio_clicks'] = user_agg['clicks'] / (user_agg['clicks'] + user_agg['carritos'] + user_agg['compras'] + user_agg['favoritos'] + 1e-10)
user_agg['ratio_compras'] = user_agg['compras'] / (user_agg['clicks'] + user_agg['carritos'] + user_agg['compras'] + user_agg['favoritos'] + 1e-10)

# Integrar información de usuarios y log al conjunto de datos principal
conjunto_datos = conjunto_datos.merge(info_usuarios, on='user_id', how='left').merge(user_agg, on='user_id', how='left')

# Separar los datos de entrenamiento y prueba
datos_entrenamiento = conjunto_datos[conjunto_datos['tipo'] == 'train'].drop(['tipo'], axis=1)
datos_prueba = conjunto_datos[conjunto_datos['tipo'] == 'test'].drop(['tipo'], axis=1)
